# Dataset
https://www.kaggle.com/mrisdal/fake-news#fake.csv

Example domain classifications (in flux) include:

  * **Fake News**: Sources that fabricate stories out of whole cloth with the intent of pranking the public.
  * **Satire**: Sources that provide humorous commentary on current events in the form of fake news.
  *  **Extreme Bias**: Sources that traffic in political propaganda and gross distortions of fact.
  *  **Conspiracy Theory**: Sources that are well-known promoters of kooky conspiracy theories.
  *  **Rumor Mill**: Sources that traffic in rumors, innuendo, and unverified claims.
  *  **State News**: Sources in repressive states operating under government sanction.
  *  **Junk Science**: Sources that promote pseudoscience, metaphysics, naturalistic fallacies, and other scientifically dubious claims.
  *  **Hate Group**: Sources that actively promote racism, misogyny, homophobia, and other forms of discrimination.
  *  **Clickbait:** Sources that are aimed at generating online advertising revenue and rely on sensationalist headlines or eye-catching pictures.
  *  **Proceed With Caution**: Sources that may be reliable but whose contents require further verification.
  *  The labels of **bs** and **junksci**, etc. do not constitute capital "t" Truth



In [103]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import mlutils
%matplotlib inline
import pandas as pd
import tensorflow_hub as hub 
import re
print tf.__version__


1.9.0


## Descarga, arregla e inspecciona el dataset Fake news

In [19]:
import os
!mkdir -p data
if not os.path.exists("dataFake/fake.csv"):
    !wget -c -nc https://github.com/chdorrego/tf/raw/master/fake-news.zip -P dataFake
    !unzip dataFake/fake-news.zip -d dataFake
    with open("dataFake/fake.csv", "r") as f:
        lines = f.readlines()
        lines = [i for i in lines if len(i.replace("\n", ""))>0]
    with open("dataFake/fake.csv", "w") as f:
        f.writelines(lines) 

--2018-09-01 08:37:15--  https://github.com/chdorrego/tf/raw/master/fake-news.zip
Resolving github.com (github.com)... 192.30.253.112, 192.30.253.113
Connecting to github.com (github.com)|192.30.253.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/chdorrego/tf/master/fake-news.zip [following]
--2018-09-01 08:37:16--  https://raw.githubusercontent.com/chdorrego/tf/master/fake-news.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.220.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.220.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21412001 (20M) [application/zip]
Saving to: ‘dataFake/fake-news.zip’

fake-news.zip       100%[===================>]  20.42M  1.87MB/s    in 13s     

2018-09-01 08:37:29 (1.57 MB/s) - ‘dataFake/fake-news.zip’ saved [21412001/21412001]

Archive:  dataFake/fake-news.zip
  inflating: dataFake/fake.csv    

In [20]:
!head -5 dataFake/fake.csv

"uuid","ord_in_thread","author","published","title","text","language","crawled","site_url","country","domain_rank","thread_title","spam_score","main_img_url","replies_count","participants_count","likes","comments","shares","type"
"6a175f46bcd24d39b3e962ad0f29936721db70db",0,"Barracuda Brigade","2016-10-26T21:41:00.000+03:00","Muslims BUSTED: They Stole Millions In Gov’t Benefits","Print They should pay all the back all the money plus interest. The entire family and everyone who came in with them need to be deported asap. Why did it take two years to bust them? 
Here we go again …another group stealing from the government and taxpayers! A group of Somalis stole over four million in government benefits over just 10 months! 
We’ve reported on numerous cases like this one where the Muslim refugees/immigrants commit fraud by scamming our system…It’s way out of control! More Related","english","2016-10-27T01:49:27.168+03:00","100percentfedup.com","US",25689,"Muslims BUSTED: They Stole Millio

In [77]:
CSV_COLUMNS  = ['uuid','ord_in_thread','author','published','title','text','language','crawled','site_url','country','domain_rank','thread_title','spam_score','main_img_url','replies_count','participants_count','likes','comments','shares','type_label']
#CSV_DEFAULTS = [[''],[0],[''],[''],[''],[''],[''],[''],[''],[''],[0],[''],[-1],[''],[-1],[-1],[-1],[-1],[-1],['']]
CSV_DEFAULTS = [[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],['']]


type_label =  ['bias','conspiracy','hate','state','junksci','satire','fake']
no_valid_labels = ['bs']

fake= pd.read_csv("dataFake/fake.csv", names = CSV_COLUMNS,skiprows=1)
print fake.shape
fake.head()

(12999, 20)


,uuid,ord_in_thread,author,published,title,text,language,crawled,site_url,country,domain_rank,thread_title,spam_score,main_img_url,replies_count,participants_count,likes,comments,shares,type_label
0,6a175f46bcd24d39b3e962ad0f29936721db70db,0,Barracuda Brigade,2016-10-26T21:41:00.000+03:00,Muslims BUSTED: They Stole Millions In Gov’t B...,Print They should pay all the back all the mon...,english,2016-10-27T01:49:27.168+03:00,100percentfedup.com,US,25689.0,Muslims BUSTED: They Stole Millions In Gov’t B...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
1,2bdc29d12605ef9cf3f09f9875040a7113be5d5b,0,reasoning with facts,2016-10-29T08:47:11.259+03:00,Re: Why Did Attorney General Loretta Lynch Ple...,Why Did Attorney General Loretta Lynch Plead T...,english,2016-10-29T08:47:11.259+03:00,100percentfedup.com,US,25689.0,Re: Why Did Attorney General Loretta Lynch Ple...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
2,c70e149fdd53de5e61c29281100b9de0ed268bc3,0,Barracuda Brigade,2016-10-31T01:41:49.479+02:00,BREAKING: Weiner Cooperating With FBI On Hilla...,Red State : \nFox News Sunday reported this mo...,english,2016-10-31T01:41:49.479+02:00,100percentfedup.com,US,25689.0,BREAKING: Weiner Cooperating With FBI On Hilla...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
3,7cf7c15731ac2a116dd7f629bd57ea468ed70284,0,Fed Up,2016-11-01T05:22:00.000+02:00,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,Email Kayla Mueller was a prisoner and torture...,english,2016-11-01T15:46:26.304+02:00,100percentfedup.com,US,25689.0,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,0.068,http://100percentfedup.com/wp-content/uploads/...,0,0,0,0,0,bias
4,0206b54719c7e241ffe0ad4315b808290dbe6c0f,0,Fed Up,2016-11-01T21:56:00.000+02:00,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,Email HEALTHCARE REFORM TO MAKE AMERICA GREAT ...,english,2016-11-01T23:59:42.266+02:00,100percentfedup.com,US,25689.0,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,0.865,http://100percentfedup.com/wp-content/uploads/...,0,0,0,0,0,bias


### Labels

* bs            10919 
* conspiracy      412 
- bias            354 
- hate            246 
- state           121 
- junksci         102 
- satire          100 
- fake             19


## Realizamos la eleminiación de la clase bs debido a la definición del problema y creamos los archivos test y train

In [144]:
d = fake.loc[fake['type_label'].isin(type_label)]
d['title'] = d['title'].str.replace('\W', ' ')
d['text'] =  d['text'].str.replace('\W', ' ')
d['thread_title'] =  d['thread_title'].str.replace('\W', '')



msk = np.random.rand(len(d)) < 0.8

d_train = d[msk==True]
d_test =  d[msk==False]

d_train.to_csv('dataFake/fake_train.csv', sep=';', encoding='utf-8', header=False)
d_test.to_csv('dataFake/fake_test.csv', sep=';', encoding='utf-8', header=False)
print d_train.count()
print d_test.count()

/opt/miniconda/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


uuid                  1231
ord_in_thread         1231
author                 836
published             1231
title                 1139
text                  1192
language              1231
crawled               1231
site_url              1231
country               1231
domain_rank           1080
thread_title          1231
spam_score            1231
main_img_url           979
replies_count         1231
participants_count    1231
likes                 1231
comments              1231
shares                1231
type_label            1231
dtype: int64
uuid                  276
ord_in_thread         276
author                192
published             276
title                 261
text                  269
language              276
crawled               276
site_url              276
country               276
domain_rank           239
thread_title          276
spam_score            276
main_img_url          214
replies_count         276
participants_count    276
likes                 276
comme

/opt/miniconda/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/miniconda/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


## Creamos la funcion para la generacion del `input pipeline`

In [170]:
#occupations = list(np.unique(d.occupation))
type_label =  ['bias','conspiracy','hate','state','junksci','satire','fake']
CSV_COLUMNS  = ['pd_rowid','uuid','ord_in_thread','author','published','title','text','language','crawled','site_url','country','domain_rank','thread_title','spam_score','main_img_url','replies_count','participants_count','likes','comments','shares','type_label']
#CSV_DEFAULTS = [[''],[0],[''],[''],[''],[''],[''],[''],[''],[''],[0],[''],[-1],[''],[-1],[-1],[-1],[-1],[-1],['']]
CSV_DEFAULTS = [[0],[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],['']]



def parse_csv(rows_string_tensor, use_features):
    columns = tf.decode_csv(rows_string_tensor, record_defaults=CSV_DEFAULTS, field_delim=";")
    features = dict(zip(CSV_COLUMNS, columns))
        
    label = features.pop("type_label")
    
    return {k:v for k,v in features.iteritems() if k in use_features}, label



def get_dataset(filename, repeat, batch_size, shuffle):
    use_features  = ['title','type_label']
    
    dataset = tf.data.TextLineDataset(filename)
    dataset = dataset.repeat(repeat)
    if shuffle:
        dataset = dataset.shuffle(buffer_size=300)
    dataset = dataset.map(lambda x: parse_csv(x, use_features))    
    dataset = dataset.batch(batch_size)
    return dataset



def get_features_defs():
    title = hub.text_embedding_column(
        key='title', 
        module_spec='https://tfhub.dev/google/random-nnlm-en-dim128/1')
        #module_spec="https://tfhub.dev/google/nnlm-en-dim128/1")
    
    return [title] 

def make_data_iterator(filename, repeat, batch_size, shuffle=True):
    
    dataset = get_dataset(filename, repeat, batch_size, shuffle)
    features_defs = get_features_defs()
    
    features, label = dataset.make_one_shot_iterator().get_next()
    
    next_X = tf.feature_column.input_layer(features, features_defs)
    next_y = label
    
    return next_X, next_y


tf.reset_default_graph()

features, label = make_data_iterator("dataFake/fake_train.csv", repeat=1, batch_size=10, shuffle=False)

with tf.Session() as sess:
    tf.tables_initializer().run()
    f,l = sess.run([features, label])   




INFO:tensorflow:Downloading TF-Hub Module 'https://tfhub.dev/google/random-nnlm-en-dim128/1'.
INFO:tensorflow:Downloaded TF-Hub Module 'https://tfhub.dev/google/random-nnlm-en-dim128/1'.
INFO:tensorflow:Initialize variable input_layer/title_hub_module_embedding/module/embeddings/part_0:0 from checkpoint /tmp/tfhub_modules/05e7c2dc95ba922793ef7aaa3ec54c8a84974024/variables/variables with embeddings


NotFoundError: Resource localhost/input_layer/title_hub_module_embedding/module/embeddings/part_0/N10tensorflow3VarE does not exist.
	 [[Node: input_layer/title_hub_module_embedding/module_apply_default/embedding_lookup_sparse/embedding_lookup = ResourceGather[Tindices=DT_INT64, dtype=DT_FLOAT, validate_indices=true, _device="/job:localhost/replica:0/task:0/device:CPU:0"](input_layer/title_hub_module_embedding/module/embeddings/part_0, input_layer/title_hub_module_embedding/module_apply_default/embedding_lookup_sparse/Unique)]]

Caused by op u'input_layer/title_hub_module_embedding/module_apply_default/embedding_lookup_sparse/embedding_lookup', defined at:
  File "/opt/miniconda/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/opt/miniconda/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/opt/miniconda/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new

### Prueba de parse_csv

In [156]:
dataset = tf.data.TextLineDataset("dataFake/fake_train.csv")
dataset = dataset.batch(2)
use_features  = ['title','type_label']
dataset = dataset.map(lambda x: parse_csv(x, use_features))     
features = dataset.make_one_shot_iterator().get_next()
with tf.Session() as sess:
    print sess.run(features)
    print sess.run(features)

({'title': array(['Muslims BUSTED  They Stole Millions In Gov   t Benefits',
       'Re  Why Did Attorney General Loretta Lynch Plead The Fifth '],
      dtype=object)}, array(['bias', 'bias'], dtype=object))
({'title': array(['BREAKING  Weiner Cooperating With FBI On Hillary Email Investigation',
       'PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnapped And Killed By ISIS   I have voted for Donald J  Trump      100percentfedUp com'],
      dtype=object)}, array(['bias', 'bias'], dtype=object))


### Prueba de la carga del dataset 

In [142]:
dataset = tf.data.TextLineDataset("dataFake/fake_train.csv")
dataset = dataset.batch(10)
features = dataset.make_one_shot_iterator().get_next()
with tf.Session() as sess:
    print sess.run(features)


['1;2bdc29d12605ef9cf3f09f9875040a7113be5d5b;0;reasoning with facts;2016-10-29T08:47:11.259+03:00;Re: Why Did Attorney General Loretta Lynch Plead The Fifth?;"Why Did Attorney General Loretta Lynch Plead The Fifth? Barracuda Brigade 2016-10-28 Print The administration is blocking congressional probe into cash payments to Iran. Of course she needs to plead the 5th. She either can\xe2\x80\x99t recall, refuses to answer, or just plain deflects the question. Straight up corruption at its finest! '
 '100percentfedUp.com ; Talk about covering your ass! Loretta Lynch did just that when she plead the Fifth to avoid incriminating herself over payments to Iran\xe2\x80\xa6Corrupt to the core! Attorney General Loretta Lynch is declining to comply with an investigation by leading members of Congress about the Obama administration\xe2\x80\x99s secret efforts to send Iran $1.7 billion in cash earlier this year, prompting accusations that Lynch has \xe2\x80\x9cpleaded the Fifth\xe2\x80\x9d Amendment t